In [85]:
import numpy as np

In [86]:
import pandas as pd
data = pd.read_csv("./data/uber_nyc_enriched.csv")

In [87]:
data.describe()

,pickups,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd
count,29101.000000,29101.000000,29101.000000,29101.000000,29101.000000,29101.000000,29101.000000,29101.000000,29101.000000,29101.000000
mean,490.215903,5.984924,8.818125,47.669042,30.823065,1017.817938,0.003830,0.026129,0.090464,2.529169
std,995.649536,3.699007,2.442897,19.814969,21.283444,7.768796,0.018933,0.093125,0.219402,4.520325
min,0.000000,0.000000,0.000000,2.000000,-16.000000,991.400000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,9.100000,32.000000,14.000000,1012.500000,0.000000,0.000000,0.000000,0.000000
50%,54.000000,6.000000,10.000000,46.000000,30.000000,1018.200000,0.000000,0.000000,0.000000,0.000000
75%,449.000000,8.000000,10.000000,64.500000,50.000000,1022.900000,0.000000,0.000000,0.050000,2.958333
max,7883.000000,21.000000,10.000000,89.000000,73.000000,1043.400000,0.280000,1.240000,2.100000,19.000000


<p> checking for nan values in dataset </p>

In [88]:
data.isnull().values.any()

True

<p> We found only Borough has nan values so we remove the nan rows </p>

In [89]:
data.isnull().any()

pickup_dt    False
borough       True
pickups      False
spd          False
vsb          False
temp         False
dewp         False
slp          False
pcp01        False
pcp06        False
pcp24        False
sd           False
hday         False
dtype: bool

In [90]:
data = data.dropna()

In [91]:
data.isnull().any()

pickup_dt    False
borough      False
pickups      False
spd          False
vsb          False
temp         False
dewp         False
slp          False
pcp01        False
pcp06        False
pcp24        False
sd           False
hday         False
dtype: bool

In [92]:
data.corr()

,pickups,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd
pickups,1.000000,0.009741,-0.008429,0.063692,0.040082,-0.015708,0.005007,-0.002821,-0.022935,-0.009676
spd,0.009741,1.000000,0.086177,-0.296126,-0.321606,-0.092761,-0.000357,0.016668,-0.010412,0.097041
vsb,-0.008429,0.086177,1.000000,0.025214,-0.231294,0.167039,-0.488407,-0.118346,0.000895,-0.047834
temp,0.063692,-0.296126,0.025214,1.000000,0.896544,-0.224537,-0.013343,-0.037295,-0.014408,-0.545558
dewp,0.040082,-0.321606,-0.231294,0.896544,1.000000,-0.311156,0.115399,0.013293,0.001519,-0.489372
slp,-0.015708,-0.092761,0.167039,-0.224537,-0.311156,1.000000,-0.089752,-0.104940,-0.134689,0.121508
pcp01,0.005007,-0.000357,-0.488407,-0.013343,0.115399,-0.089752,1.000000,0.128064,0.000997,0.000310
pcp06,-0.002821,0.016668,-0.118346,-0.037295,0.013293,-0.104940,0.128064,1.000000,0.251166,0.039943
pcp24,-0.022935,-0.010412,0.000895,-0.014408,0.001519,-0.134689,0.000997,0.251166,1.000000,0.069664
sd,-0.009676,0.097041,-0.047834,-0.545558,-0.489372,0.121508,0.000310,0.039943,0.069664,1.000000


In [93]:
data

,pickup_dt,borough,pickups,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd,hday
0,2015-01-01 01:00:00,Bronx,152,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
1,2015-01-01 01:00:00,Brooklyn,1519,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
2,2015-01-01 01:00:00,EWR,0,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
3,2015-01-01 01:00:00,Manhattan,5258,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
4,2015-01-01 01:00:00,Queens,405,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
5,2015-01-01 01:00:00,Staten Island,6,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
7,2015-01-01 02:00:00,Bronx,120,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0,Y
8,2015-01-01 02:00:00,Brooklyn,1229,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0,Y
9,2015-01-01 02:00:00,EWR,0,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0,Y
10,2015-01-01 02:00:00,Manhattan,4345,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0,Y


In [94]:
del data['dewp']

In [95]:
data

,pickup_dt,borough,pickups,spd,vsb,temp,slp,pcp01,pcp06,pcp24,sd,hday
0,2015-01-01 01:00:00,Bronx,152,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,Y
1,2015-01-01 01:00:00,Brooklyn,1519,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,Y
2,2015-01-01 01:00:00,EWR,0,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,Y
3,2015-01-01 01:00:00,Manhattan,5258,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,Y
4,2015-01-01 01:00:00,Queens,405,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,Y
5,2015-01-01 01:00:00,Staten Island,6,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,Y
7,2015-01-01 02:00:00,Bronx,120,3.0,10.0,30.0,1023.0,0.0,0.0,0.0,0.0,Y
8,2015-01-01 02:00:00,Brooklyn,1229,3.0,10.0,30.0,1023.0,0.0,0.0,0.0,0.0,Y
9,2015-01-01 02:00:00,EWR,0,3.0,10.0,30.0,1023.0,0.0,0.0,0.0,0.0,Y
10,2015-01-01 02:00:00,Manhattan,4345,3.0,10.0,30.0,1023.0,0.0,0.0,0.0,0.0,Y


In [96]:
data['hday'] = data['hday'].apply(lambda x: 0 if x=='N' else 1)

In [97]:
oneHotCols = pd.get_dummies(data['borough'])
data = data.join(oneHotCols)

In [98]:
data

,pickup_dt,borough,pickups,spd,vsb,temp,slp,pcp01,pcp06,pcp24,sd,hday,Bronx,Brooklyn,EWR,Manhattan,Queens,Staten Island
0,2015-01-01 01:00:00,Bronx,152,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,1,1,0,0,0,0,0
1,2015-01-01 01:00:00,Brooklyn,1519,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,1,0,1,0,0,0,0
2,2015-01-01 01:00:00,EWR,0,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,1,0,0,1,0,0,0
3,2015-01-01 01:00:00,Manhattan,5258,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0
4,2015-01-01 01:00:00,Queens,405,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,1,0,0,0,0,1,0
5,2015-01-01 01:00:00,Staten Island,6,5.0,10.0,30.0,1023.5,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1
7,2015-01-01 02:00:00,Bronx,120,3.0,10.0,30.0,1023.0,0.0,0.0,0.0,0.0,1,1,0,0,0,0,0
8,2015-01-01 02:00:00,Brooklyn,1229,3.0,10.0,30.0,1023.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,0
9,2015-01-01 02:00:00,EWR,0,3.0,10.0,30.0,1023.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0,0
10,2015-01-01 02:00:00,Manhattan,4345,3.0,10.0,30.0,1023.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0


In [99]:
del data['borough']

### 1 hot encoding the time of day

<p> We based our hour groups by sunrise and sunset. Night is the time when the sun is down, which on average is from 8pm to 6am. Morning is from 6am till noon. Afternoon is from noon till 5pm. Evening is from 5pm till 8pm, which is around when the sunsets. 

In [100]:
dateTest = data['pickup_dt'][0]
print(int(dateTest[11:13]))
data['is_morning'] = data['pickup_dt'].apply(lambda x: 1 if (int(x[11:13]) >= 6 and int(x[11:13]) < 12) else 0)
data['is_afternoon'] = data['pickup_dt'].apply(lambda x: 1 if (int(x[11:13]) >= 12 and int(x[11:13]) < 17) else 0)
data['is_evening'] = data['pickup_dt'].apply(lambda x: 1 if (int(x[11:13]) >= 17 and int(x[11:13]) < 21) else 0)
data['is_night']  = data['pickup_dt'].apply(lambda x: 1 if (int(x[11:13]) >= 21 or int(x[11:13]) < 6) else 0)

1


### 1 hot encoding the weekday

The weekday from the pickup_dt feature has been 1 hot encoded into monday-sunday. We believe having each day as a feature will help classify & predict the number of ubers necessary at a future date

In [101]:
import datetime
data['is_monday'] = data['pickup_dt'].apply(lambda x: 1 if datetime.date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday() == 0 else 0)
data['is_tuesday'] = data['pickup_dt'].apply(lambda x: 1 if datetime.date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday() == 1 else 0)
data['is_wednesday'] = data['pickup_dt'].apply(lambda x: 1 if datetime.date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday() == 2 else 0)
data['is_thursday'] = data['pickup_dt'].apply(lambda x: 1 if datetime.date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday() == 3 else 0)
data['is_friday'] = data['pickup_dt'].apply(lambda x: 1 if datetime.date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday() == 4 else 0)
data['is_saturday'] = data['pickup_dt'].apply(lambda x: 1 if datetime.date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday() == 5 else 0)
data['is_sunday'] = data['pickup_dt'].apply(lambda x: 1 if datetime.date(int(x[0:4]),int(x[5:7]),int(x[8:10])).weekday() == 6 else 0)

In [102]:
del data['pickup_dt']

<p> We found that the borough EWR averages about 2.4 pickups every 96 hours so we are getting rid of the EWR borough from our dataset. We found that the borough Staten Island averages 1.6 pickups and hour and had a max 13 pickups in an hour over 6 months so we got rid of it from our dataset. </p>

In [103]:
d1 = data.where(data['EWR']==1)[['pickups','EWR']]
print(d1.describe())
d1 = d1.dropna()
data = data[data.EWR == 0]
del data['EWR']
d1 = data.where(data['Staten Island']==1)[['pickups','Staten Island']]
print(d1.describe())
d1 = d1.dropna()
data = data[data['Staten Island'] == 0]
del data['Staten Island']

           pickups     EWR
count  4343.000000  4343.0
mean      0.024177     1.0
std       0.160937     0.0
min       0.000000     1.0
25%       0.000000     1.0
50%       0.000000     1.0
75%       0.000000     1.0
max       2.000000     1.0
           pickups  Staten Island
count  4343.000000         4343.0
mean      1.601888            1.0
std       1.640451            0.0
min       0.000000            1.0
25%       0.000000            1.0
50%       1.000000            1.0
75%       2.000000            1.0
max      13.000000            1.0


In [104]:
data.describe()

,pickups,spd,vsb,temp,slp,pcp01,pcp06,pcp24,sd,hday,...,is_afternoon,is_evening,is_night,is_monday,is_tuesday,is_wednesday,is_thursday,is_friday,is_saturday,is_sunday
count,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,...,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000,17372.000000
mean,820.426606,6.000039,8.820027,47.489005,1017.812802,0.003821,0.026074,0.091036,2.536496,0.038453,...,0.208381,0.166705,0.374856,0.143679,0.143679,0.138153,0.143449,0.143679,0.143679,0.143679
std,1179.027565,3.706364,2.443018,19.772345,7.783546,0.018834,0.092917,0.220437,4.520593,0.192292,...,0.406163,0.372723,0.484100,0.350775,0.350775,0.345071,0.350540,0.350775,0.350775,0.350775
min,0.000000,0.000000,0.000000,2.000000,991.400000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,86.000000,3.000000,9.100000,31.500000,1012.400000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,367.000000,6.000000,10.000000,45.000000,1018.200000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,789.250000,8.000000,10.000000,64.000000,1023.000000,0.000000,0.000000,0.051667,3.166667,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7883.000000,21.000000,10.000000,89.000000,1043.400000,0.280000,1.240000,2.100000,19.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


<p> We have three cateogries of pickup traffic low, medium, high. 

In [105]:
Man = data[data.Manhattan == 1]
Bronx = data[data.Bronx == 1]
Queens = data[data.Queens == 1]
Brooklyn = data[data.Brooklyn == 1]

print(Man['pickups'].describe())
print(Bronx['pickups'].describe())
print(Queens['pickups'].describe())
print(Brooklyn['pickups'].describe())

count    4343.000000
mean     2387.253281
std      1434.724668
min         0.000000
25%      1223.500000
50%      2269.000000
75%      3293.500000
max      7883.000000
Name: pickups, dtype: float64
count    4343.000000
mean       50.667050
std        31.029223
min         0.000000
25%        29.000000
50%        46.000000
75%        66.000000
max       262.000000
Name: pickups, dtype: float64
count    4343.000000
mean      309.354824
std       154.368300
min         0.000000
25%       196.000000
50%       308.000000
75%       410.000000
max       831.000000
Name: pickups, dtype: float64
count    4343.000000
mean      534.431269
std       294.810182
min         0.000000
25%       331.500000
50%       493.000000
75%       675.000000
max      2009.000000
Name: pickups, dtype: float64


In [106]:
mstd = Man['pickups'].std()
mmean = Man['pickups'].mean()
Man['pickupPrediction'] = Man['pickups'].apply(lambda x: 0 if x < (mmean - mstd/2) else (
                                               2 if (x > (mmean + mstd/2))
                                                else 1))
print(Man['pickupPrediction'].describe())

mstd = Bronx['pickups'].std()
mmean = Bronx['pickups'].mean()
Bronx['pickupPrediction'] = Bronx['pickups'].apply(lambda x: 0 if x < (mmean - mstd/2) else (
                                               2 if (x > (mmean + mstd/2))
                                                else 1))
print(Bronx['pickupPrediction'].describe())

mstd = Queens['pickups'].std()
mmean = Queens['pickups'].mean()
Queens['pickupPrediction'] = Queens['pickups'].apply(lambda x: 0 if x < (mmean - mstd/2) else (
                                               2 if (x > (mmean + mstd/2))
                                                else 1))
print(Queens['pickupPrediction'].describe())

mstd = Brooklyn['pickups'].std()
mmean = Brooklyn['pickups'].mean()
Brooklyn['pickupPrediction'] = Brooklyn['pickups'].apply(lambda x: 0 if x < (mmean - mstd/2) else (
                                               2 if (x > (mmean + mstd/2))
                                                else 1))
print(Brooklyn['pickupPrediction'].describe())

count    4343.000000
mean        0.953028
std         0.774480
min         0.000000
25%         0.000000
50%         1.000000
75%         2.000000
max         2.000000
Name: pickupPrediction, dtype: float64


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

count    4343.000000
mean        0.907898
std         0.757453
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: pickupPrediction, dtype: float64
count    4343.000000
mean        0.980428
std         0.794867
min         0.000000
25%         0.000000
50%         1.000000
75%         2.000000
max         2.000000
Name: pickupPrediction, dtype: float64
count    4343.000000
mean        0.901911
std         0.757005
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: pickupPrediction, dtype: float64


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [107]:
from sklearn.model_selection import train_test_split
yMan = Man['pickupPrediction']
del Man['pickupPrediction']
yBronx = Bronx['pickupPrediction']
del Bronx['pickupPrediction']
yQueens = Queens['pickupPrediction']
del Queens['pickupPrediction']
yBrooklyn = Brooklyn['pickupPrediction']
del Brooklyn['pickupPrediction']

In [108]:
xManTrain, xManTest, yManTrain, YManTest = train_test_split(Man, yMan, test_size = 1/5, random_state = 0)
xBronxTrain, xBronxTest, yBronxTrain, YBronxTest = train_test_split(Bronx, yBronx, test_size = 1/5, random_state = 0)
xQueensTrain, xQueensTest, yQueensTrain, YQueensTest = train_test_split(Queens, yQueens, test_size = 1/5, random_state = 0)
xBrooklynTrain, xBrooklynTest, yBrooklynTrain, YBrooklynTest = train_test_split(Brooklyn, yBrooklyn, test_size = 1/5, random_state = 0)
xTrain = pd.concat([xManTrain,xBronxTrain,xQueensTrain,xBrooklynTrain])
xTest = pd.concat([xManTest,xBronxTest,xQueensTest,xBrooklynTest])
yTrain = pd.concat([yManTrain,yBronxTrain,yQueensTrain,yBrooklynTrain])
yTest = pd.concat([YManTest,YBronxTest,YQueensTest,YBrooklynTest])

In [109]:
from sklearn.preprocessing import StandardScaler

xTrain.reset_index(inplace=True, drop=True)
yTrain.reset_index(inplace=True, drop=True)
xTest.reset_index(inplace=True, drop=True)
yTest.reset_index(inplace=True, drop=True)
xTrain = xTrain.values
yTrain = yTrain.values
xTest = xTest.values
yTest = yTest.values
xTrain = StandardScaler().fit(xTrain).transform(xTrain)
XTest = StandardScaler().fit(xTest).transform(xTest)

In [110]:
from scipy.special import expit
class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001,regul='L2'):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.regul=regul
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        if(self.regul == 'L1'):
            gradient[1:] += -2 * np.sign(self.w_[1:]) * self.C
        elif(self.regul == 'L2'):
            gradient[1:] += -2 * self.w_[1:] * self.C
        else:
            gradient[1:] += (-2 * self.w_[1:]+ np.sign(self.w_[1:])) * self.C
        return gradient
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 
            # add bacause maximizing 

In [111]:
class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation 
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        if(self.regul == 'L1'):
            gradient[1:] += -2 * np.sign(self.w_[1:]) * self.C
        elif(self.regul == 'L2'):
            gradient[1:] += -2 * self.w_[1:] * self.C
        else:
            gradient[1:] += (-2 * self.w_[1:]+ np.sign(self.w_[1:])) * self.C
        
        return gradient

In [112]:
from scipy.optimize import minimize_scalar
import copy
class LineSearchLogisticRegression(BinaryLogisticRegression):
    
    # define custom line search for problem
    @staticmethod
    def objective_function(eta,X,y,w,grad,C,regul):
        wnew = w - grad*eta
        g = expit(X @ wnew)
        if(regul=='L1'):
            wobj = -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + C*sum(abs(wnew))
        elif(regul=='L2'):
            wobj = -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + C*sum(wnew**2)
        elif(regul=='both'):
            wobj = -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + C*sum(abs(wnew)+wnew**2)
        return wobj
    
        
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = -self._get_gradient(Xb,y)
            # minimization inopposite direction
            
            # do line search in gradient direction, using scipy function
            opts = {'maxiter':self.iters/50} # unclear exactly what this should be
            res = minimize_scalar(self.objective_function, # objective function to optimize
                                  bounds=(self.eta/1000,self.eta*10), #bounds to optimize
                                  args=(Xb,y,self.w_,gradient,self.C,self.regul), # additional argument for objective function
                                  method='bounded', # bounded optimization for speed
                                  options=opts) # set max iterations
            
            eta = res.x # get optimal learning rate
            self.w_ -= gradient*eta # set new function values
            # subtract to minimize
                

In [113]:
from scipy.optimize import fmin_bfgs
class BFGSBinaryLogisticRegression(BinaryLogisticRegression):
    
    @staticmethod
    def objective_function(w,X,y,C,regul):
        g = expit(X @ w)
        return -np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C,regul):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        if(regul == 'L1'):
            gradient[1:] += -2 * np.sign(w[1:]) * C
        elif(regul == 'L2'):
            gradient[1:] += -2 * w[1:] * C
        else:
            gradient[1:] += (-2 * w[1:]+ np.sign(w[1:])) * C
        return -gradient
    
    # just overwrite fit function
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C,self.regul), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)
        
        self.w_ = self.w_.reshape((num_features,1))

In [114]:
class LogisticRegression:
    def __init__(self, eta, iterations=20,method='none',C=0.01, regul='L2'):
        self.eta = eta
        self.iters = iterations
        self.method = method
        self.C = C
        self.regul = regul
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            if(self.method == 'none'):
                blr = BinaryLogisticRegression(self.eta,self.iters)
            elif(self.method == 'stoc'):
                blr = StochasticLogisticRegression(self.eta,self.iters,self.C)
            elif(self.method=='steep'):
                blr=LineSearchLogisticRegression(self.eta,self.iters,self.C,self.regul)
            elif(self.method=='newt'):
                blr = BFGSBinaryLogisticRegression(self.eta,self.iters,self.C,self.regul)

            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row

In [115]:
from sklearn.metrics import accuracy_score
lr = LogisticRegression(0.1,500)
%time lr.fit(xTrain,yTrain)

yhat = lr.predict(xTest)
print('Accuracy of: ',accuracy_score(yTest,yhat))

CPU times: user 5.45 s, sys: 1.87 s, total: 7.32 s
Wall time: 3.77 s
Accuracy of:  0.2827963176064442


## steepest descent

### 'L1','L2', or 'both' for regularization

In [116]:
lr = LogisticRegression(0.01,100,'steep',0.001,'both')
%time lr.fit(xTrain,yTrain)

yhat = lr.predict(xTest)
print('Accuracy of: ',accuracy_score(yTest,yhat))

CPU times: user 1.93 s, sys: 473 ms, total: 2.41 s
Wall time: 1.21 s
Accuracy of:  0.3722669735327963


## stochastic-gradient descent

In [117]:
lr = LogisticRegression(0.1,1000,'stoc',0.01,'both')
%time lr.fit(xTrain,yTrain)

yhat = lr.predict(xTest)
print('Accuracy of: ',accuracy_score(yTest,yhat))

CPU times: user 154 ms, sys: 23 ms, total: 177 ms
Wall time: 92.6 ms
Accuracy of:  0.3613348676639816


## quasi newton method

In [118]:
lr = LogisticRegression(0.01,50,'newt',0.1,'L1')
%time lr.fit(xTrain,yTrain)

yhat = lr.predict(xTest)
print('Accuracy of: ',accuracy_score(yTest,yhat))

CPU times: user 540 ms, sys: 147 ms, total: 687 ms
Wall time: 376 ms
Accuracy of:  0.3731300345224396


In [119]:
# %%time
# # how do we compare now to sklearn?
# from sklearn.linear_model import LogisticRegression

# lr_sk = LogisticRegression(solver='lbfgs',n_jobs=1) # all params default
# # note that sklearn is optimized for using the liblinear library with logistic regression
# # ...and its faster than our implementation here
# lr_sk.fit(xTrain,yTrain) # no need to add bias term, sklearn does it internally!!
# print(lr_sk.coef_)
# yhat = lr_sk.predict(xTest)
# print('Accuracy of: ',accuracy_score(yTest,yhat))

## Grid Search

In [162]:
techniques = ['none','steep','stoc','newt']
vals = [10,1,0.1,0.01,0.001,0.0001]
maxNone = [0,0]
maxSteep = [0,0]
maxStoc = [0,0]
maxNewt = [0,0]
maxVal = [0,0,'type']
noneX = []
noneY = []
steepX = []
steepY = []
stocX = []
stocY = []
newtX = []
newtY = []
for tech in techniques:
    for val in vals:
        lr = LogisticRegression(0.01,50,tech,val,'L1')
        lr.fit(xTrain,yTrain)
        yhat = lr.predict(xTest)
        score = accuracy_score(yTest,yhat)
        if(tech == 'none'):
            noneX.append(val)
            noneY.append(score)
            if(score > maxNone[0]):
                maxNone[0] = score
                maxNone[1] = val
        if(tech == 'steep'):
            steepX.append(val)
            steepY.append(score)
            if(score > maxSteep[0]):
                maxSteep[0] = score
                maxSteep[1] = val
        if(tech == 'stoc'):
            stocX.append(val)
            stocY.append(score)
            if(score > maxStoc[0]):
                maxStoc[0] = score
                maxStoc[1] = val
        if(tech == 'newt'):
            newtX.append(val)
            newtY.append(score)
            if(score > maxNewt[0]):
                maxNewt[0] = score
                maxNewt[1] = val
        if(score > maxVal[0]):
            maxVal[0] = score
            maxVal[1] = val
            maxVal[2] = tech

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:

In [163]:
print('max for regular lr', maxNone)
print('max for steep', maxSteep)
print('max for stocastic', maxStoc)
print('max for newton\'s', maxNewt)
print('max overall', maxVal)

max for regular lr [0.33630609896432684, 10]
max for steep [0.4919447640966628, 1]
max for stocastic [0.4076524741081703, 1]
max for newton's [0.3731300345224396, 0.1]
max overall [0.4919447640966628, 1, 'steep']


In [164]:
import matplotlib.pyplot as plt
%matplotlib inline

In [177]:
print(noneX)
print(noneY)
my_xticks = ['10','1','0.1','0.01','0.001','0.0001']
plt.set_xticklabels(my_xticks, rotation='horizontal', fontsize=18)
plt.title('Stocastic')
plt.plot(stocX,stocY)

[10, 1, 0.1, 0.01, 0.001, 0.0001]
[0.33630609896432684, 0.33630609896432684, 0.33630609896432684, 0.33630609896432684, 0.33630609896432684, 0.33630609896432684]


AttributeError: module 'matplotlib.pyplot' has no attribute 'set_xticklabels'